In [1]:
# these modules are requried to import my code
import  sys
import  os
from    pathlib import Path

# this adds my code to the system path so it can be imported as if it were a python module
cwd         = os.getcwd()
geograph_interpolator_root  = Path(cwd).__str__()
sys.path.append(geograph_interpolator_root)

# import our package
from geograph_super_pred import *

c:\Users\22566465\Miniconda3\envs\ginn_gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

basement_file = r'Data\survey_data.csv'

df = pd.read_csv(basement_file)
df.z = df.z - df.z.max()
df.z = df.z *10

d_frame = df

buffer = 10
x       = d_frame.x
y       = d_frame.y

spline = vd.Spline(damping=0.0008)
spline.fit((d_frame['x'],d_frame['y']), d_frame['z'])

grid_coords     = vd.grid_coordinates(region=(d_frame['x'].min()-buffer, d_frame['x'].max()+buffer,d_frame['y'].min()-buffer, d_frame['y'].max()+buffer), spacing=50)
gridded_scalars = spline.predict(grid_coords)

In [11]:
core_node_list = []

for s in stations:
    core_node_list.extend(s.conv2core())

visualise_3D(core_node_list,'rdata')


30

In [13]:
a_stations = generate_synth_stations(df,100,vert_res=20, feat_type='A')
b_stations = generate_synth_stations(df,100,vert_res=50, feat_type='B')

In [4]:
def generate_synth_stations(griddf, num_stations, vert_res=30, feat_type='A'):

    # for finding basement value
    buffer = 10
    x       = griddf.x
    y       = griddf.y

    spline = vd.Spline(damping=0.0008)
    spline.fit((griddf['x'],griddf['y']), griddf['z'])

    grid_coords     = vd.grid_coordinates(region=(griddf['x'].min()-buffer, griddf['x'].max()+buffer,griddf['y'].min()-buffer, griddf['y'].max()+buffer), spacing=50)
    gridded_scalars = spline.predict(grid_coords)

    x_array = [random.randint(griddf.x.min(),griddf.x.max()) for v in range(num_stations)]
    y_array = [random.randint(griddf.y.min(),griddf.y.max()) for v in range(num_stations)]

    station_list = []

    station_z = np.linspace(0,griddf.z.min(),vert_res)

    for i in range(num_stations):

        # TODO you would sample the basement location from the grid based on randomly generated xy
        basement_depth = spline.predict((float(x_array[i]),float(y_array[i]),0))

        points_above = [x for x in station_z if x>basement_depth]
        points_below = [x for x in station_z if x<=basement_depth]

        # feature_type A
        if feat_type=='A':
            feats_above = [np.sin(x) for x in np.linspace(0,3,len(points_above))]
            feats_below = [(np.sin(x)*-1)/2 for x in np.linspace(0,3,len(points_below))]

        # feature_type B
        if feat_type=='B':
            feats_above = list(np.random.choice(np.linspace(-0.02, 0.02,100), len(points_above), replace=False))
            feats_below = list(np.random.choice(np.linspace(-1, 1, 100), len(points_below), replace=False))
        
        loc = [x_array[i],y_array[i],basement_depth]
        params = {'depth':station_z,'rdata':feats_above+feats_below,'label':[0]*len(points_above)+[1]*len(points_below)}

        station_list.append(Station(i,loc,feat_type,params))

    station_values = feats_above + feats_below

    return(station_list)

In [ ]:

import numpy as np
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt 
  
## generate the data and plot it for an ideal normal curve

  
## x-axis for the plot
x_data = np.linspace(0,1,0.1)
  
## y-axis as the gaussian
y_data = stats.norm.pdf(x_data, 0.5, 1)
  
## plot data
plt.plot(x_data, y_data)

In [ ]:
def generate_stations(feature_type, grid_df, number_of_stations=100, vertical_resolution=30, ):


def populate_station():

In [ ]:
# load the repo demo data
node_file_name      = geograph_interpolator_root+'\Data\graph_features.csv'
edge_file_name_sim  = geograph_interpolator_root+'\Data\graph_edges_similar.csv'
edge_file_name_lat  = geograph_interpolator_root+'\Data\graph_edges_lattice.csv'

core_node_list  = load_graph_nodes(node_file_name)
lattice_edges   = load_graph_edges(edge_file_name_lat)
similar_edges   = load_graph_edges(edge_file_name_sim)

In [ ]:
# visualise the different node parameters a list can be seen here: core_node_list[0].params.keys()
# role - either training or testing
# labl - label
# cond - pretend conductivity
visualise_3D(core_node_list,'cond',similar_edges)

In [ ]:
# declare global variables for later visualisation
global graph
global gcn_logits
global wgcnlogits
global gat_logits

feats = ['data','zloc_n'] # feats can be a list of keys to extract from graph.ndata or number for random embedding
insize = len(feats) if isinstance(feats,list) else feats

# construct our graph objects
graph = create_dgl_graph(core_node_list,lattice_edges)
graph0 = create_dgl_graph(core_node_list,lattice_edges)
graph1 = create_dgl_graph(core_node_list,lattice_edges)
graph2 = create_dgl_graph(core_node_list,lattice_edges)

# generate our GNN Models
model_GCN = GCN(insize,5,2)
modelwGCN = wGCN(insize,5,2)
model_GAT = GAT(insize,5,2,5)

# run interpolation using the different models
gcn_loss, gcn_logits = gnn_interpolate(model_GCN, graph0, node_feats=feats, epochs=300)
wgcnloss, wgcnlogits = gnn_interpolate(modelwGCN, graph1, node_feats=feats, epochs=300)
gat_loss, gat_logits = gnn_interpolate(model_GAT, graph2, node_feats=feats, epochs=300)

In [ ]:
# visualise training loss
plt.title('Loss Value')
plt.plot(gat_loss,label='GAT')
plt.plot(gcn_loss,label='GCN')
plt.plot(wgcnloss,label='wGCN')
plt.legend()

In [ ]:
# generates a prediction and opens a pyvista slider to move around results
plotter = pv.Plotter(shape=(1, 3),notebook=False,window_size=(1920,1000))

def create_mesh(value):
    res = int(value)

    x = graph.ndata['xloc'].flatten().detach().numpy()
    y = graph.ndata['yloc'].flatten().detach().numpy()
    z = graph.ndata['zloc'].flatten().detach().numpy()
    p0 = torch.argmax(gcn_logits[res],dim=1)
    p1 = torch.argmax(wgcnlogits[res],dim=1)
    p2 = torch.argmax(gat_logits[res],dim=1)

    # wow a pyvista object
    vert = list(zip(x,y,z))

    gcn_cloud = pv.PolyData(vert)
    gcn_cloud['pred'] = p0    

    wgcncloud = pv.PolyData(vert)
    wgcncloud['pred'] = p1  

    gat_cloud = pv.PolyData(vert)
    gat_cloud['pred'] = p2   

    # GCN
    plotter.subplot(0, 0)
    plotter.add_text("GCN", font_size=30)
    plotter.add_mesh(gcn_cloud,name='gcn_pointcloud')
    
    # wGCN
    plotter.subplot(0, 1)
    plotter.add_text("wGCN", font_size=30)
    plotter.add_mesh(wgcncloud,name='wgcnpointcloud')

    # GAT
    plotter.subplot(0, 2)
    plotter.add_text("GAT", font_size=30)
    plotter.add_mesh(gat_cloud, name='gat_pointcloud')

    return


plotter.add_slider_widget(create_mesh, [0, 299], title='Epoch')
plotter.show()